#ingest races file

###1. read data from csv

In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DoubleType,DateType

In [0]:
races_schema=StructType(fields=[StructField("raceId",IntegerType(),False),
                                StructField("year",IntegerType(),True),
                                StructField("round",IntegerType(),True),
                                StructField("circuitId",IntegerType(),True),
                                StructField("name",StringType(),True),
                                StructField("date",DateType(),True),
                                StructField("time",StringType(),True),
                                StructField("url",StringType(),True)])

In [0]:
races_df=spark.read \
              .option("header",True) \
              .schema(races_schema) \
              .csv("abfss://raw@formula1dlbtg.dfs.core.windows.net/races.csv")

In [0]:
display(races_df)

In [0]:
races_df.printSchema()

##2. add races timestamp and ingestion date

In [0]:
from pyspark.sql.functions import current_timestamp,to_timestamp,concat,lit,col

In [0]:
races_with_timestamp_df=races_df.withColumn("ingestion_date",current_timestamp()).withColumn("race_timestamp",to_timestamp(concat(col("date"),lit(" "),col("time")), "yyyy-MM-dd HH:mm:ss"))
display(races_with_timestamp_df)

###3. races selected df

In [0]:
races_selected_df = races_with_timestamp_df.select(col("raceId").alias("race_id"),col("year").alias("race_year"),col("round"),col("circuitId").alias("circuit_id"),col("name"),col("ingestion_date"),col("race_timestamp"))

In [0]:
display(races_selected_df)

In [0]:
races_selected_df.write.mode("overwrite").partitionBy("race_year").parquet("abfss://processed@formula1dlbtg.dfs.core.windows.net/races")

In [0]:
df=spark.read.parquet("abfss://processed@formula1dlbtg.dfs.core.windows.net/races")
display(df)

In [0]:
display(spark.read.parquet("abfss://processed@formula1dlbtg.dfs.core.windows.net/races"))